# 📊 **Uso del Web Scraping para la Obtención de Número de Seguidores de Cuentas de X** 🚀

Este proyecto se centra en la obtención y análisis de los datos de seguidores de cuentas en X utilizando técnicas de web scraping.

---

## 🧹 **1. Limpieza de Datos - `limpiar.py`**

En esta sección, realizamos la limpieza de los datos obtenidos a través del web scraping. El código de la función `limpiar.py` se encarga de eliminar valores nulos, duplicados y corregir cualquier inconsistencia en los datos para asegurar un análisis preciso.

```python
# Código de limpiar.py aquí


## 📊 **2. Visualización de Datos - `graficos.py`** 🎨

En esta sección, creamos gráficos para representar las tendencias de los seguidores de las cuentas de X. Utilizamos diversas herramientas de visualización para facilitar el análisis de los datos obtenidos en la etapa anterior.

- **Objetivo**: Generar gráficos interactivos y estáticos que muestren las relaciones entre los seguidores y otras variables relevantes, como el tiempo, la actividad de las cuentas, etc.

### Gráficos Generados:
1. **Gráfico de Seguidores a lo Largo del Tiempo**: Muestra la evolución de seguidores de las cuentas de X.
2. **Gráfico de Distribución de Seguidores**: Representa la distribución de los seguidores entre las diferentes cuentas.

```python
# Código de graficos.py aquí


## 📈 **3. Análisis Exploratorio de Datos - `aed.py`** 🔍

En esta sección, realizamos un análisis exhaustivo de los datos obtenidos para entender patrones, relaciones y comportamientos clave. El análisis exploratorio de datos (AED) nos ayuda a identificar tendencias y correlaciones significativas entre las diferentes variables.

- **Objetivo**: Explorar los datos mediante estadísticas descriptivas, análisis de correlación y técnicas de detección de anomalías para extraer conclusiones valiosas que guíen la toma de decisiones.

### Técnicas Utilizadas:
1. **Estadísticas Descriptivas**: Calculo de métricas clave como la media, mediana, desviación estándar y otros valores estadísticos que nos permiten tener una primera comprensión de los datos.
2. **Correlación entre Variables**: Investigamos las relaciones entre las variables, como el número de seguidores y la actividad de las cuentas.
3. **Detección de Outliers**: Identificamos registros atípicos o fuera de lo común en los datos, lo que puede indicar comportamientos interesantes o errores en el scraping.

```python
# Código de aed.py aquí
